# Improved K-Means Clustering: Skill-Based Feature Engineering
## Objective: Generate high-quality job similarity mapping for the Streamlit app

This notebook improves clustering by:
1. Using **skill-based features** instead of noisy free-text
2. Testing multiple K values (20–80) with evaluation metrics
3. Manually inspecting clusters for semantic coherence
4. Implementing post-filtering logic for better "similar opportunities"
5. Creating a compact mapping file for the app

In [1]:
import pandas as pd
import numpy as np
import ast
from collections import Counter
import time
import warnings
import pickle
from pathlib import Path

# ML and clustering
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import normalize
from scipy.sparse import csr_matrix, vstack
import joblib

warnings.filterwarnings('ignore')

print("=" * 80)
print("📚 IMPROVED K-MEANS CLUSTERING FOR SKILL-BASED JOB SIMILARITY")
print("=" * 80)
print(f"\n✅ All libraries loaded successfully!")


📚 IMPROVED K-MEANS CLUSTERING FOR SKILL-BASED JOB SIMILARITY

✅ All libraries loaded successfully!


## Section 1: Load and Explore Skill-Based Features

Load the processed jobs file and verify skill_list availability.

In [2]:
# Load data
data_path = "/kaggle/input/job-skills/all_jobs_mapped.csv"
print(f"Loading from: {data_path}")

df = pd.read_csv(data_path)
print(f"\n📊 Dataset shape: {df.shape}")
print(f"📋 Columns: {list(df.columns)}")

# Create a stable job_id alias from job_key (if not already present)
if 'job_key' in df.columns and 'job_id' not in df.columns:
    df['job_id'] = df['job_key'].astype(str)

# Check for required columns
required_cols = ['job_id', 'job_title', 'skill_list', 'location', 'company', 'skill_categories']
for col in required_cols:
    if col in df.columns:
        print(f"  ✅ {col}")
    else:
        print(f"  ❌ {col} - MISSING")

# Parse skill_list column
def parse_skill_list(x):
    """Convert skill_list from string representation to actual list."""
    if isinstance(x, list):
        return x
    if pd.isna(x):
        return []
    try:
        val = ast.literal_eval(str(x))
        if isinstance(val, list):
            return [str(s).strip().lower() for s in val]
    except:
        pass
    return []

df['skill_list'] = df['skill_list'].apply(parse_skill_list)

# Display stats
print(f"\n📈 Skill Statistics:")
print(f"  Total jobs: {len(df)}")
print(f"  Jobs with skills: {df['skill_list'].apply(len).gt(0).sum()}")
print(f"  Avg skills per job: {df['skill_list'].apply(len).mean():.1f}")
print(f"  Max skills in a job: {df['skill_list'].apply(len).max()}")

# Show samples
print(f"\n🎯 Sample Jobs:")
for idx in df.sample(min(3, len(df)), random_state=42).index:
    job = df.iloc[idx]
    print(f"\n  Job: {job['job_title']}")
    print(f"  Location: {job['location']}")
    print(f"  Skills ({len(job['skill_list'])}): {job['skill_list'][:5]}...")


Loading from: /kaggle/input/job-skills/all_jobs_mapped.csv

📊 Dataset shape: (2884556, 9)
📋 Columns: ['source', 'job_key', 'job_title', 'job_description', 'skills_raw', 'location', 'company', 'skill_list', 'skill_categories']
  ✅ job_id
  ✅ job_title
  ✅ skill_list
  ✅ location
  ✅ company
  ✅ skill_categories

📈 Skill Statistics:
  Total jobs: 2884556
  Jobs with skills: 2881535
  Avg skills per job: 10.4
  Max skills in a job: 463

🎯 Sample Jobs:

  Job: Air Freight Manager in Slough, UK
  Location: Slough, England, United Kingdom
  Skills (7): ['air freight management', 'import', 'export', 'time management', 'financial management']...

  Job: Investment Analyst
  Location: Tehran
  Skills (1): ['investment management financial analysis risk assessment asset allocation portfolio optimization']...

  Job: Supply Chain Manager
  Location: Wellington
  Skills (1): ['demand forecasting inventory management data analysis sales and operations planning (s&op) erp software proficiency']...


## Section 2: Build Sparse Skill Vector Representation

Create a binary skill matrix (1 if skill present, 0 otherwise) using only the most frequent skills to reduce noise.

In [3]:
# Build skill vocabulary: only include skills appearing in >= min_freq jobs
min_freq = 30  # Adjust based on data size
all_skills = Counter()

for skills in df['skill_list']:
    all_skills.update(skills)

# Keep only frequent skills
skill_vocab = sorted([skill for skill, count in all_skills.most_common() if count >= min_freq])
print(f"📊 Skill Vocabulary:")
print(f"  Total unique skills: {len(all_skills)}")
print(f"  Frequent skills (>= {min_freq} occurrences): {len(skill_vocab)}")
print(f"  Top 20 skills by frequency:")
for skill, count in all_skills.most_common(20):
    print(f"    {skill:40} {count:5} jobs")

# Build binary skill matrix: rows=jobs, cols=skills
def build_skill_matrix(df, skill_vocab):
    """Build sparse binary matrix where X[i,j] = 1 if job i has skill j."""
    n_jobs = len(df)
    n_skills = len(skill_vocab)
    skill_to_idx = {skill: idx for idx, skill in enumerate(skill_vocab)}
    
    print(f"\n🔨 Building sparse skill matrix ({n_jobs} jobs × {n_skills} skills)...")
    
    rows, cols, data = [], [], []
    
    for job_idx, skills in enumerate(df['skill_list']):
        for skill in skills:
            if skill in skill_to_idx:
                rows.append(job_idx)
                cols.append(skill_to_idx[skill])
                data.append(1)
    
    X_sparse = csr_matrix((data, (rows, cols)), shape=(n_jobs, n_skills), dtype=np.int8)
    
    print(f"✅ Sparse matrix created: shape {X_sparse.shape}, density {X_sparse.nnz / (n_jobs * n_skills) * 100:.2f}%")
    return X_sparse, skill_vocab

X_sparse, skill_vocab = build_skill_matrix(df, skill_vocab)

# Keep a copy of the original dataframe for later reference
df_original = df.copy()


📊 Skill Vocabulary:
  Total unique skills: 2707646
  Frequent skills (>= 30 occurrences): 61954
  Top 20 skills by frequency:
    communication                            367386 jobs
    customer service                         275711 jobs
    teamwork                                 224816 jobs
    communication skills                     194746 jobs
    leadership                               185228 jobs
    problem solving                          148894 jobs
    time management                          141447 jobs
    attention to detail                      132686 jobs
    problemsolving                           128635 jobs
    project management                       121936 jobs
    interpersonal skills                     99280 jobs
    patient care                             98982 jobs
    sales                                    93515 jobs
    nursing                                  87518 jobs
    collaboration                            86685 jobs
    training            

## Section 3: Tune K-Means with Multiple K Values

Test K values from 20 to 80 and compute inertia + silhouette scores on a sample.

In [4]:
# Tune K-Means for multiple K values
k_values = [20, 40, 60, 80]
tuning_results = []

# Use a sample for silhouette scoring (large datasets can be slow)
sample_size = min(5000, len(df))
sample_indices = np.random.choice(len(df), sample_size, replace=False)
X_sample = X_sparse[sample_indices]

print(f"\n🎯 K-MEANS TUNING")
print(f"Testing K values: {k_values}")
print(f"Using sample of {sample_size} jobs for silhouette score evaluation\n")

start_time = time.time()

for k in k_values:
    print(f"Training K-Means with K={k}...")
    
    # Train on full dataset
    kmeans = MiniBatchKMeans(
        n_clusters=k,
        batch_size=1000,
        random_state=42,
        n_init=3,
        max_iter=100
    )
    labels_full = kmeans.fit_predict(X_sparse)
    
    # Compute metrics
    inertia = kmeans.inertia_
    silhouette = silhouette_score(X_sample, kmeans.predict(X_sample), sample_size=1000)
    
    # Store results
    tuning_results.append({
        'K': k,
        'Inertia': inertia,
        'Silhouette': silhouette,
        'Model': kmeans,
        'Labels': labels_full
    })
    
    print(f"  ✅ K={k:2d} | Inertia: {inertia:12.0f} | Silhouette: {silhouette:.4f}\n")

elapsed = time.time() - start_time
print(f"⏱️  Tuning completed in {elapsed/60:.1f} minutes")

# Convert to DataFrame for easy comparison
tuning_df = pd.DataFrame(tuning_results)
print("\n📊 K-Means Tuning Summary:")
print(tuning_df[['K', 'Inertia', 'Silhouette']].to_string(index=False))



🎯 K-MEANS TUNING
Testing K values: [20, 40, 60, 80]
Using sample of 5000 jobs for silhouette score evaluation

Training K-Means with K=20...
  ✅ K=20 | Inertia:     23763101 | Silhouette: -0.2560

Training K-Means with K=40...
  ✅ K=40 | Inertia:     22898509 | Silhouette: -0.1073

Training K-Means with K=60...
  ✅ K=60 | Inertia:     23602078 | Silhouette: -0.2288

Training K-Means with K=80...
  ✅ K=80 | Inertia:     22547104 | Silhouette: -0.0247

⏱️  Tuning completed in 0.6 minutes

📊 K-Means Tuning Summary:
 K      Inertia  Silhouette
20 2.376310e+07   -0.256014
40 2.289851e+07   -0.107254
60 2.360208e+07   -0.228800
80 2.254710e+07   -0.024728


## Section 4: Evaluate Clusters with Inertia and Silhouette Score

Visualize and select the optimal K based on metrics and manual inspection.

In [5]:
print("\n" + "="*80)
print("📈 K-MEANS EVALUATION METRICS")
print("="*80)

# Normalize metrics for comparison
tuning_df['Inertia_Norm'] = (tuning_df['Inertia'] - tuning_df['Inertia'].min()) / (tuning_df['Inertia'].max() - tuning_df['Inertia'].min())
tuning_df['Silhouette_Norm'] = (tuning_df['Silhouette'] - tuning_df['Silhouette'].min()) / (tuning_df['Silhouette'].max() - tuning_df['Silhouette'].min())

print("\nMetrics Comparison (normalized 0-1):")
print(tuning_df[['K', 'Inertia_Norm', 'Silhouette_Norm']].to_string(index=False))

# Find best K by each metric
best_by_silhouette = tuning_df.loc[tuning_df['Silhouette'].idxmax(), 'K']
best_by_inertia = tuning_df.loc[tuning_df['Inertia'].idxmin(), 'K']

print(f"\n🏆 Best by Silhouette Score: K={int(best_by_silhouette)}")
print(f"🏆 Best by Inertia (Elbow): K={int(best_by_inertia)}")
print(f"\n💡 Recommendation: Check cluster quality manually and select K based on semantic coherence.")



📈 K-MEANS EVALUATION METRICS

Metrics Comparison (normalized 0-1):
 K  Inertia_Norm  Silhouette_Norm
20      1.000000         0.000000
40      0.288986         0.643186
60      0.867579         0.117665
80      0.000000         1.000000

🏆 Best by Silhouette Score: K=80
🏆 Best by Inertia (Elbow): K=80

💡 Recommendation: Check cluster quality manually and select K based on semantic coherence.


## Section 5: Inspect Cluster Quality by Job Titles and Skills

For each K value, inspect representative clusters to assess semantic coherence.

In [6]:
def inspect_clusters(df, labels, skill_vocab, X_sparse, k, num_samples=3):
    """Inspect cluster quality for a given K value."""
    print(f"\n{'='*80}")
    print(f"CLUSTER INSPECTION: K={k}")
    print(f"{'='*80}")
    
    df_temp = df_original.copy()
    df_temp['cluster'] = labels
    
    # Get cluster sizes
    cluster_sizes = df_temp['cluster'].value_counts().sort_index()
    print(f"\n📊 Cluster Size Distribution:")
    print(f"  Min: {cluster_sizes.min():5d} jobs")
    print(f"  Max: {cluster_sizes.max():5d} jobs")
    print(f"  Mean: {cluster_sizes.mean():5.0f} jobs")
    print(f"  Std: {cluster_sizes.std():5.0f}")
    
    # Sample clusters to inspect
    sampled_clusters = np.random.choice(k, num_samples, replace=False)
    
    for cluster_id in sorted(sampled_clusters):
        cluster_jobs = df_temp[df_temp['cluster'] == cluster_id]
        cluster_mask = (df_temp['cluster'] == cluster_id).values
        X_cluster = X_sparse[cluster_mask]
        
        print(f"\n🔍 Cluster {cluster_id} ({len(cluster_jobs)} jobs)")
        print(f"  {'─'*76}")
        
        # Top job titles
        print(f"  Top 10 Job Titles:")
        top_jobs = cluster_jobs['job_title'].value_counts().head(10)
        for job, count in top_jobs.items():
            print(f"    • {job:50} ({count:3d} occurrences)")
        
        # Top skills in this cluster
        print(f"\n  Top 20 Most Frequent Skills:")
        cluster_skills = Counter()
        for skills in cluster_jobs['skill_list']:
            cluster_skills.update(skills)
        
        for skill, count in cluster_skills.most_common(20):
            pct = (count / len(cluster_jobs)) * 100
            print(f"    • {skill:40} {count:4d} jobs ({pct:5.1f}%)")
        
        # Dominant skill categories (if available)
        if 'skill_categories' in df_temp.columns:
            print(f"\n  Skill Categories:")
            cats = cluster_jobs['skill_categories'].fillna('').value_counts().head(5)
            for cat, count in cats.items():
                if cat:
                    print(f"    • {cat}: {count} jobs")

# Inspect clusters for a few K values to assess quality
for result in tuning_results[:2]:  # Inspect K=20 and K=40 for now
    k = result['K']
    labels = result['Labels']
    inspect_clusters(df_original, labels, skill_vocab, X_sparse, k, num_samples=2)



CLUSTER INSPECTION: K=20

📊 Cluster Size Distribution:
  Min:     1 jobs
  Max: 2452415 jobs
  Mean: 144228 jobs
  Std: 550181

🔍 Cluster 12 (31 jobs)
  ────────────────────────────────────────────────────────────────────────────
  Top 10 Job Titles:
    • Mental Health Counselor                            (  2 occurrences)
    • Permanent Charge Nurse (RN) - Acute Care of the Elderly, ACE Unit (  1 occurrences)
    • Spécialiste en activités cliniques                 (  1 occurrences)
    • Social Work Care Manager                           (  1 occurrences)
    • Senior Manager, Product Manager, Innovation Lab    (  1 occurrences)
    • Registered Nurse (I, II, III, or IV), Acute Care - Inpatient Pediatric Unit at Bassett Healthcare (  1 occurrences)
    • Registered Nurse -(All vacancies including PRN, PT, FT) at Veterans Affairs, Veterans Health Administration (  1 occurrences)
    • Program Supervisor                                 (  1 occurrences)
    • Mental Health Therapist

## Section 6: Select Optimal K and Re-fit Model

Based on silhouette scores, inertia, and manual inspection, select the best K and train final model.

In [7]:
# SELECT OPTIMAL K
# Based on silhouette score and manual inspection, select the best K
# You can adjust this based on your inspection results above

optimal_k = 40  # Adjust based on silhouette score and cluster quality inspection

print("\n" + "="*80)
print(f"✅ SELECTED OPTIMAL K = {optimal_k}")
print("="*80)

# Get the model with optimal K
optimal_result = next(r for r in tuning_results if r['K'] == optimal_k)
optimal_kmeans = optimal_result['Model']
optimal_labels = optimal_result['Labels']

# Verify cluster distribution
unique, counts = np.unique(optimal_labels, return_counts=True)
print(f"\n📊 Final Cluster Distribution (K={optimal_k}):")
print(f"  Clusters: {len(unique)}")
print(f"  Min cluster size: {counts.min()}")
print(f"  Max cluster size: {counts.max()}")
print(f"  Mean cluster size: {counts.mean():.0f}")
print(f"  Std cluster size: {counts.std():.0f}")

# Add cluster labels to the original dataframe
df_original['cluster_id'] = optimal_labels

print(f"\n✅ Cluster assignments added to dataframe")



✅ SELECTED OPTIMAL K = 40

📊 Final Cluster Distribution (K=40):
  Clusters: 38
  Min cluster size: 5
  Max cluster size: 856670
  Mean cluster size: 75909
  Std cluster size: 157637

✅ Cluster assignments added to dataframe


## Section 7: Create Compact Cluster Mapping

Create a lightweight mapping file with job_id, title, company, location, cluster_id, and main_category for the Streamlit app.

In [8]:
# Function to extract dominant skill category per job
def get_main_category(skill_categories_str):
    """Extract primary skill category from the concatenated string."""
    if pd.isna(skill_categories_str) or not skill_categories_str:
        return 'other'
    categories = str(skill_categories_str).split(',')
    return categories[0].strip() if categories else 'other'

# Build compact mapping
print("\n" + "="*80)
print("📦 CREATING COMPACT CLUSTER MAPPING")
print("="*80)

# df_original contains job_id thanks to Section 1
cluster_mapping = df_original[[
    'job_id', 'job_title', 'company', 'location', 'cluster_id'
]].copy()

cluster_mapping['main_category'] = df_original['skill_categories'].apply(get_main_category)

print(f"\n📊 Compact Mapping Shape: {cluster_mapping.shape}")
print(f"📋 Columns: {list(cluster_mapping.columns)}")
print(f"\n🎯 Sample Records:")
print(cluster_mapping.head(10).to_string(index=False))

# Save as pickle (more efficient than CSV for repeated loading)
output_path = "/kaggle/working/job_clusters_small_v2.pkl"
cluster_mapping.to_pickle(output_path)
print(f"\n✅ Saved: {output_path}")

# Also save as CSV for debugging/inspection
csv_path = output_path.replace('.pkl', '.csv')
cluster_mapping.to_csv(csv_path, index=False)
print(f"✅ Saved: {csv_path}")

# File size comparison
import os
pkl_size = os.path.getsize(output_path) / 1024 / 1024
csv_size = os.path.getsize(csv_path) / 1024 / 1024
print(f"\n💾 File Sizes:")
print(f"  Pickle: {pkl_size:.2f} MB")
print(f"  CSV: {csv_size:.2f} MB")



📦 CREATING COMPACT CLUSTER MAPPING

📊 Compact Mapping Shape: (2884556, 6)
📋 Columns: ['job_id', 'job_title', 'company', 'location', 'cluster_id', 'main_category']

🎯 Sample Records:
                                                                                                                job_id                               job_title                                 company                                             location  cluster_id   main_category
                  https://ae.linkedin.com/jobs/view/bar-supervisor-saadiyat-beach-club-at-aldar-hospitality-3795826819    Bar Supervisor - Saadiyat Beach Club                       Aldar Hospitality   Abu Dhabi, Abu Dhabi Emirate, United Arab Emirates          24           other
                       https://ae.linkedin.com/jobs/view/duty-engineer-at-arjaan-hotel-apartments-by-rotana-3801219786                           Duty Engineer       Arjaan Hotel Apartments by Rotana              Abu Dhabi Emirate, United Arab Emirates      

## Section 8: Implement Post-Filter Logic for Similar Jobs

Define helper functions to extract dominant skills and filter similar jobs by skill overlap.

In [9]:
print("\n" + "="*80)
print("🔧 SKILL-BASED FILTERING LOGIC")
print("="*80)

# Extract dominant skills per job for filtering
def get_top_skills(skill_list, n_top=5):
    """Get top N most common skills."""
    return skill_list[:n_top] if skill_list else []

# Create a skills lookup table for filtering
skills_lookup = pd.DataFrame({
    'job_id': df_original['job_id'],
    'top_skills': df_original['skill_list'].apply(lambda x: get_top_skills(x, n_top=5)),
    'skill_categories': df_original['skill_categories'],
    'location': df_original['location'],
    'cluster_id': df_original['cluster_id']
})

def get_skill_overlap(skills_a, skills_b):
    """Compute skill overlap between two job skill lists."""
    set_a = set(skills_a)
    set_b = set(skills_b)
    overlap = set_a & set_b
    return len(overlap), overlap

def filter_similar_jobs(target_job_id, cluster_jobs_df, df_with_skills, min_skill_overlap=1):
    """
    Filter similar jobs based on:
    1. Same cluster
    2. At least min_skill_overlap skills in common
    3. Optional: same skill category

    Returns: filtered DataFrame and match_quality flag
    """
    # Get target job skills
    target_job = df_with_skills[df_with_skills['job_id'] == str(target_job_id)]
    if target_job.empty:
        return pd.DataFrame(), 'not_found'

    target_skills = target_job.iloc[0]['top_skills']
    target_category = target_job.iloc[0]['skill_categories']

    # Filter candidates by skill overlap
    filtered = []
    for _, candidate in cluster_jobs_df.iterrows():
        if candidate['job_id'] == str(target_job_id):
            continue  # Skip the target job itself

        candidate_skills = candidate['top_skills']
        overlap, _ = get_skill_overlap(target_skills, candidate_skills)

        if overlap >= min_skill_overlap:
            filtered.append(candidate)

    if filtered:
        return pd.DataFrame(filtered), 'high_quality'
    else:
        # Fallback: return unfiltered cluster jobs (loose match)
        return cluster_jobs_df[cluster_jobs_df['job_id'] != str(target_job_id)], 'loose_match'

print("\n✅ Filtering functions created")
print("\n💡 Filtering Strategy:")
print("   1. Find jobs in same cluster")
print("   2. Filter by >= 1 overlapping key skill")
print("   3. Fallback to unfiltered cluster if needed (marked as 'loose_match')")

# Test filtering with a sample nursing job
sample_nursing = df_original[df_original['job_title'].str.contains('nurse', case=False, na=False)]
if not sample_nursing.empty:
    test_job_id = sample_nursing.iloc[0]['job_id']
    test_job_title = sample_nursing.iloc[0]['job_title']
    test_cluster = sample_nursing.iloc[0]['cluster_id']

    print(f"\n🧪 TEST: Filtering similar jobs for '{test_job_title}'")
    print(f"   Job ID: {test_job_id}")
    print(f"   Cluster: {test_cluster}")

    # Get cluster jobs
    cluster_jobs = skills_lookup[skills_lookup['cluster_id'] == test_cluster]

    # Apply filtering
    filtered_jobs, match_quality = filter_similar_jobs(
        test_job_id,
        cluster_jobs,
        skills_lookup,
        min_skill_overlap=1
    )

    print(f"\n   Original cluster size: {len(cluster_jobs)}")
    print(f"   After filtering: {len(filtered_jobs)}")
    print(f"   Match quality: {match_quality}")

    if not filtered_jobs.empty:
        print(f"\n   Filtered sample jobs:")
        for idx, job in filtered_jobs.head(3).iterrows():
            match_row = cluster_mapping[cluster_mapping['job_id'] == job['job_id']]
            title = match_row['job_title'].values[0] if not match_row.empty else 'Unknown'
            print(f"     • {title}")



🔧 SKILL-BASED FILTERING LOGIC

✅ Filtering functions created

💡 Filtering Strategy:
   1. Find jobs in same cluster
   2. Filter by >= 1 overlapping key skill
   3. Fallback to unfiltered cluster if needed (marked as 'loose_match')

🧪 TEST: Filtering similar jobs for 'Associate Nurse Unit Manager - Custodial FYMHS'
   Job ID: https://au.linkedin.com/jobs/view/%09associate-nurse-unit-manager-custodial-fymhs-at-the-royal-melbourne-hospital-3803879583
   Cluster: 34

   Original cluster size: 213959
   After filtering: 1158
   Match quality: high_quality

   Filtered sample jobs:
     • Audit Senior
     • Case Manager
     • Case Manager


## Section 9: Export Results and Save for App Integration

Save the fitted K-Means model, cluster mapping, and skills lookup for use in the Streamlit app.

In [10]:
print("\n" + "="*80)
print("💾 SAVING RESULTS FOR STREAMLIT APP")
print("="*80)

# 1. Save the fitted K-Means model
kmeans_model_path = "/kaggle/working/kmeans_model_v2.pkl"
joblib.dump({
    'model': optimal_kmeans,
    'n_clusters': optimal_k,
    'skill_vocab': skill_vocab,
    'skill_to_idx': {skill: idx for idx, skill in enumerate(skill_vocab)},
    'min_freq': min_freq,
}, kmeans_model_path)
print(f"✅ K-Means model: {kmeans_model_path}")

# 2. Save the cluster mapping (already saved earlier)
print(f"✅ Cluster mapping: ../data/processed/job_clusters_small_v2.pkl")

# 3. Save skills lookup for filtering
skills_lookup_path = "/kaggle/working/skills_lookup_v2.pkl"
skills_lookup.to_pickle(skills_lookup_path)
print(f"✅ Skills lookup: {skills_lookup_path}")

# 4. Save filtering functions as a module reference
print(f"\n✅ Filtering logic ready for src/models/cluster_analyzer.py")

print(f"\n" + "="*80)
print(f"📊 FINAL SUMMARY")
print(f"="*80)
print(f"  Total jobs clustered: {len(cluster_mapping)}")
print(f"  Number of clusters (K): {optimal_k}")
print(f"  Avg jobs per cluster: {len(cluster_mapping) / optimal_k:.0f}")
print(f"  Skill vocabulary size: {len(skill_vocab)}")
print(f"  Feature matrix: {X_sparse.shape}")
print(f"\n  Saved Files:")
print(f"    • job_clusters_small_v2.pkl (cluster mapping)")
print(f"    • job_clusters_small_v2.csv (for inspection)")
print(f"    • kmeans_model_v2.pkl (fitted model)")
print(f"    • skills_lookup_v2.pkl (for filtering)")
print(f"\n✅ CLUSTERING PIPELINE COMPLETE!")
print(f"   Next: Update src/models/cluster_analyzer.py to load v2 files")



💾 SAVING RESULTS FOR STREAMLIT APP
✅ K-Means model: /kaggle/working/kmeans_model_v2.pkl
✅ Cluster mapping: ../data/processed/job_clusters_small_v2.pkl
✅ Skills lookup: /kaggle/working/skills_lookup_v2.pkl

✅ Filtering logic ready for src/models/cluster_analyzer.py

📊 FINAL SUMMARY
  Total jobs clustered: 2884556
  Number of clusters (K): 40
  Avg jobs per cluster: 72114
  Skill vocabulary size: 61954
  Feature matrix: (2884556, 61954)

  Saved Files:
    • job_clusters_small_v2.pkl (cluster mapping)
    • job_clusters_small_v2.csv (for inspection)
    • kmeans_model_v2.pkl (fitted model)
    • skills_lookup_v2.pkl (for filtering)

✅ CLUSTERING PIPELINE COMPLETE!
   Next: Update src/models/cluster_analyzer.py to load v2 files
